# Bài 2: Biến đổi Cosine (Cosine Transform)

## Yêu cầu:
- Hiểu và áp dụng biến đổi cosine
- Sử dụng DCT (Discrete Cosine Transform) trong xử lý dữ liệu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler

## 1. Biến đổi Cosine cơ bản

In [ ]:
# Tạo tín hiệu mẫu
N = 100
t = np.linspace(0, 1, N)
signal = np.sin(2 * np.pi * 5 * t) + 0.5 * np.sin(2 * np.pi * 10 * t)

# Vẽ tín hiệu gốc
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.plot(t, signal)
plt.title('Tín hiệu gốc')
plt.xlabel('Thời gian')
plt.ylabel('Biên độ')
plt.grid(True)

## 2. Áp dụng DCT

In [ ]:
# Biến đổi DCT
dct_coeffs = dct(signal, norm='ortho')

# Vẽ hệ số DCT
plt.subplot(1, 3, 2)
plt.plot(dct_coeffs)
plt.title('Hệ số DCT')
plt.xlabel('Tần số')
plt.ylabel('Hệ số')
plt.grid(True)

## 3. Nén dữ liệu bằng DCT

In [ ]:
# Giữ lại 20% hệ số lớn nhất
keep_ratio = 0.2
n_keep = int(N * keep_ratio)

# Tạo bản sao và loại bỏ các hệ số nhỏ
dct_compressed = dct_coeffs.copy()
dct_compressed[n_keep:] = 0

# Biến đổi ngược
signal_reconstructed = idct(dct_compressed, norm='ortho')

# Vẽ tín hiệu phục hồi
plt.subplot(1, 3, 3)
plt.plot(t, signal, label='Gốc', alpha=0.7)
plt.plot(t, signal_reconstructed, label='Phục hồi', linestyle='--')
plt.title(f'Tín hiệu phục hồi ({keep_ratio*100}% hệ số)')
plt.xlabel('Thời gian')
plt.ylabel('Biên độ')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Tính sai số
mse = np.mean((signal - signal_reconstructed)**2)
print(f"\nMean Squared Error: {mse:.6f}")
print(f"Tỷ lệ nén: {(1 - keep_ratio) * 100:.1f}%")

## 4. Áp dụng DCT cho ảnh

In [ ]:
# Load dữ liệu digits
digits = load_digits()
sample_image = digits.images[0]

# Áp dụng DCT 2D
dct_image = dct(dct(sample_image.T, norm='ortho').T, norm='ortho')

# Vẽ so sánh
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

axes[0].imshow(sample_image, cmap='gray')
axes[0].set_title('Ảnh gốc')
axes[0].axis('off')

axes[1].imshow(np.log(np.abs(dct_image) + 1), cmap='hot')
axes[1].set_title('Hệ số DCT (log scale)')
axes[1].axis('off')

# Nén ảnh
keep_ratio = 0.1
threshold = np.percentile(np.abs(dct_image), (1 - keep_ratio) * 100)
dct_compressed = dct_image.copy()
dct_compressed[np.abs(dct_compressed) < threshold] = 0

# Phục hồi ảnh
image_reconstructed = idct(idct(dct_compressed.T, norm='ortho').T, norm='ortho')

axes[2].imshow(image_reconstructed, cmap='gray')
axes[2].set_title(f'Ảnh phục hồi ({keep_ratio*100}% hệ số)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

# Tính sai số
mse = np.mean((sample_image - image_reconstructed)**2)
print(f"\nMean Squared Error: {mse:.6f}")
print(f"Số hệ số khác 0: {np.count_nonzero(dct_compressed)} / {dct_image.size}")

## 5. DCT cho trích xuất đặc trưng

In [ ]:
# Hàm trích xuất đặc trưng DCT
def extract_dct_features(image, n_coeffs=20):
    """
    Trích xuất n_coeffs hệ số DCT đầu tiên từ ảnh
    """
    dct_2d = dct(dct(image.T, norm='ortho').T, norm='ortho')
    # Lấy các hệ số theo thứ tự zigzag (đơn giản hóa: lấy từ góc trên trái)
    features = []
    for i in range(int(np.sqrt(n_coeffs)) + 1):
        for j in range(int(np.sqrt(n_coeffs)) + 1):
            if len(features) < n_coeffs:
                features.append(dct_2d[i, j])
    return np.array(features)

# Áp dụng cho dataset digits
n_samples = 100
n_coeffs = 20

X_dct = np.array([extract_dct_features(img, n_coeffs) for img in digits.images[:n_samples]])
y = digits.target[:n_samples]

print(f"\nShape của đặc trưng DCT: {X_dct.shape}")
print(f"Shape gốc: {digits.images[:n_samples].reshape(n_samples, -1).shape}")
print(f"Giảm số chiều: {digits.images[0].size} -> {n_coeffs}")

## Bài tập thực hành

1. Thử nghiệm với các tỷ lệ nén khác nhau (10%, 20%, 50%) và so sánh MSE
2. Áp dụng DCT cho toàn bộ dataset digits và huấn luyện một mô hình phân loại
3. So sánh hiệu quả của DCT với PCA trong việc giảm chiều dữ liệu
4. Giải thích tại sao DCT hiệu quả trong nén dữ liệu
5. Tìm hiểu về ứng dụng của DCT trong JPEG compression